In [ ]:
!pip install -q transformers accelerate bitsandbytes nltk unsloth pandas matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.5/218.5 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺ 154.2/156.5 MB 211.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# LLM Code Translation Evaluation
# Evaluating unsloth/Llama-3.2-3B-Instruct-bnb-4bit on code translation tasks

# Step 1: Setup Environment
!pip install -q transformers accelerate bitsandbytes nltk unsloth pandas matplotlib

# Import necessary libraries
import os
import json
import time
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from unsloth import FastLanguageModel

# Download NLTK data for BLEU calculation
nltk.download('punkt')

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 2: Load the Model and Tokenizer
def load_model():
    # Configure quantization
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    # Load model and tokenizer with unsloth for faster inference
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
        trust_remote_code=True,
        device_map="auto",
        quantization_config=bnb_config,
    )

    # Set model to evaluation mode
    model = model.eval()

    return model, tokenizer

# Step 3: Define CodeBLEU functions
# A simplified CodeBLEU implementation that focuses on basic aspects
def calculate_bleu(reference, candidate):
    """Calculate BLEU score between reference and candidate."""
    # Tokenize reference and candidate
    reference_tokens = nltk.word_tokenize(reference)
    candidate_tokens = nltk.word_tokenize(candidate)

    # Calculate BLEU score with smoothing
    smooth = SmoothingFunction().method1

    # Calculate with different n-gram weights
    weights_1 = (1.0, 0, 0, 0)  # BLEU-1
    weights_2 = (0.5, 0.5, 0, 0)  # BLEU-2
    weights_4 = (0.25, 0.25, 0.25, 0.25)  # BLEU-4

    try:
        bleu_1 = sentence_bleu([reference_tokens], candidate_tokens, weights=weights_1, smoothing_function=smooth)
        bleu_2 = sentence_bleu([reference_tokens], candidate_tokens, weights=weights_2, smoothing_function=smooth)
        bleu_4 = sentence_bleu([reference_tokens], candidate_tokens, weights=weights_4, smoothing_function=smooth)
        return bleu_1, bleu_2, bleu_4
    except Exception as e:
        print(f"Error calculating BLEU: {e}")
        return 0, 0, 0

def calculate_keyword_match(reference, candidate, language):
    """Calculate keyword matching score based on language keywords."""
    # Define keywords for each language
    keywords = {
        'python': ['def', 'class', 'if', 'elif', 'else', 'for', 'while', 'try', 'except',
                  'import', 'from', 'return', 'with', 'as', 'True', 'False', 'None',
                  'and', 'or', 'not', 'in', 'is', 'lambda'],
        'java': ['public', 'private', 'protected', 'class', 'interface', 'extends', 'implements',
                'void', 'int', 'boolean', 'String', 'if', 'else', 'for', 'while', 'try', 'catch',
                'return', 'new', 'this', 'super', 'static', 'final', 'null', 'true', 'false'],
        'cpp': ['int', 'float', 'double', 'char', 'bool', 'void', 'class', 'struct', 'enum',
               'const', 'static', 'if', 'else', 'for', 'while', 'switch', 'case', 'break',
               'return', 'try', 'catch', 'throw', 'new', 'delete', 'public', 'private', 'protected']
    }

    if language not in keywords:
        print(f"Warning: Keywords not defined for {language}")
        return 0

    # Tokenize reference and candidate
    ref_tokens = set(nltk.word_tokenize(reference))
    cand_tokens = set(nltk.word_tokenize(candidate))

    # Find language keywords in reference and candidate
    ref_keywords = set(ref_tokens.intersection(keywords[language]))
    cand_keywords = set(cand_tokens.intersection(keywords[language]))

    # Calculate F1 score for keyword matching
    if not ref_keywords:
        return 0

    precision = len(ref_keywords.intersection(cand_keywords)) / len(cand_keywords) if cand_keywords else 0
    recall = len(ref_keywords.intersection(cand_keywords)) / len(ref_keywords) if ref_keywords else 0

    if precision + recall == 0:
        return 0

    f1 = 2 * precision * recall / (precision + recall)
    return f1

def calculate_codebleu(reference, candidate, target_language):
    """Calculate a simplified version of CodeBLEU."""
    # Calculate BLEU scores
    bleu_1, bleu_2, bleu_4 = calculate_bleu(reference, candidate)

    # Calculate keyword matching score
    keyword_match = calculate_keyword_match(reference, candidate, target_language)

    # Weighted sum (simplified CodeBLEU)
    # 0.7 * BLEU + 0.3 * keyword_match
    codebleu = 0.7 * bleu_4 + 0.3 * keyword_match

    return {
        'bleu_1': bleu_1,
        'bleu_2': bleu_2,
        'bleu_4': bleu_4,
        'keyword_match': keyword_match,
        'codebleu': codebleu
    }


def load_dataset_from_file(file_path):
    """
    Load a CSV with columns: problem_id, C++, Java, Python
    and build a dict:
      {
        'cpp_to_java':   [ { 'source_code': ..., 'target_code': ... }, ... ],
        'cpp_to_python': [ ... ],
        'java_to_cpp':   [ ... ],
        'java_to_python':[ ... ],
        'python_to_cpp': [ ... ],
        'python_to_java':[ ... ],
      }
    """
    df = pd.read_csv(file_path)
    # map our lowercase keys to DataFrame column names
    col_map = {'cpp': 'C++', 'java': 'Java', 'python': 'Python'}
    dataset = {}

    langs = ['cpp', 'java', 'python']
    for src in langs:
        for tgt in langs:
            if src == tgt:
                continue
            key = f"{src}_to_{tgt}"
            examples = []
            for _, row in df.iterrows():
                examples.append({
                    'source_code': row[col_map[src]],
                    'target_code': row[col_map[tgt]]
                })
            dataset[key] = examples[:10]

    return dataset

# Step 5: Generate code translations using the LLM
def generate_translation(model, tokenizer, source_code, source_lang, target_lang, max_new_tokens=1024):
    """Generate code translation using the LLM."""
    prompt = f"""Translate the following {source_lang} code to {target_lang}:

```{source_lang}
{source_code}
```

Translated {target_lang} code:
"""

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate translation
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.1,  # Lower temperature for more deterministic output
            num_beams=1,      # Greedy decoding
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the translated code part
    translated_code = generated_text.split("Translated " + target_lang + " code:")[1].strip()

    # Remove any markdown code blocks if present
    if translated_code.startswith("```"):
        translated_code = translated_code.split("```")[1]
        if target_lang.lower() in translated_code.lower().split("\n")[0]:
            translated_code = "\n".join(translated_code.split("\n")[1:])

    # Remove ending code block if present
    if "```" in translated_code:
        translated_code = translated_code.split("```")[0]

    return translated_code.strip()

# Step 6: Evaluate translations
def evaluate_translations(model, tokenizer, dataset):
    """Evaluate the model on all translation directions."""
    results = {}

    # Define the language pairs
    language_pairs = [
        ('java', 'python'),
        ('java', 'cpp'),
        ('python', 'java'),
        ('python', 'cpp'),
        ('cpp', 'java'),
        ('cpp', 'python')
    ]

    # Run evaluations for each pair
    for source_lang, target_lang in language_pairs:
        pair_key = f"{source_lang}_to_{target_lang}"
        print(f"\nEvaluating {pair_key}...")

        pair_results = []

        # Skip if no examples for this pair
        if pair_key not in dataset or not dataset[pair_key]:
            print(f"No examples found for {pair_key}")
            continue

        # Process each example
        for i, example in enumerate(tqdm(dataset[pair_key])):
            source_code = example['source_code']
            reference_code = example['target_code']

            # Generate translation
            try:
                start_time = time.time()
                translated_code = generate_translation(
                    model, tokenizer, source_code, source_lang, target_lang
                )
                inference_time = time.time() - start_time

                # Calculate metrics
                metrics = calculate_codebleu(reference_code, translated_code, target_lang)

                # Store results
                pair_results.append({
                    'example_id': i,
                    'source_code': source_code,
                    'reference_code': reference_code,
                    'translated_code': translated_code,
                    'bleu_1': metrics['bleu_1'],
                    'bleu_2': metrics['bleu_2'],
                    'bleu_4': metrics['bleu_4'],
                    'keyword_match': metrics['keyword_match'],
                    'codebleu': metrics['codebleu'],
                    'inference_time': inference_time
                })

                print(f"Example {i} - CodeBLEU: {metrics['codebleu']:.4f}")

            except Exception as e:
                print(f"Error processing example {i}: {e}")

        # Store results for this pair
        results[pair_key] = pair_results

    return results

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 6.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.7.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.0 which is incompatible.


<ipython-input-3-9e761651108e>:19: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using device: cuda


In [ ]:
# Step 7: Analyze results
def analyze_results(results):
    """Analyze and visualize evaluation results."""
    # Calculate average metrics for each translation direction
    summary = {}

    for pair, pair_results in results.items():
        if not pair_results:
            continue

        avg_bleu_1 = np.mean([r['bleu_1'] for r in pair_results])
        avg_bleu_2 = np.mean([r['bleu_2'] for r in pair_results])
        avg_bleu_4 = np.mean([r['bleu_4'] for r in pair_results])
        avg_keyword = np.mean([r['keyword_match'] for r in pair_results])
        avg_codebleu = np.mean([r['codebleu'] for r in pair_results])
        avg_time = np.mean([r['inference_time'] for r in pair_results])

        summary[pair] = {
            'avg_bleu_1': avg_bleu_1,
            'avg_bleu_2': avg_bleu_2,
            'avg_bleu_4': avg_bleu_4,
            'avg_keyword_match': avg_keyword,
            'avg_codebleu': avg_codebleu,
            'avg_inference_time': avg_time,
            'num_examples': len(pair_results)
        }

    # Create a summary dataframe
    df_summary = pd.DataFrame({
        'Translation': list(summary.keys()),
        'Avg BLEU-1': [summary[k]['avg_bleu_1'] for k in summary],
        'Avg BLEU-2': [summary[k]['avg_bleu_2'] for k in summary],
        'Avg BLEU-4': [summary[k]['avg_bleu_4'] for k in summary],
        'Avg Keyword Match': [summary[k]['avg_keyword_match'] for k in summary],
        'Avg CodeBLEU': [summary[k]['avg_codebleu'] for k in summary],
        'Avg Inference Time (s)': [summary[k]['avg_inference_time'] for k in summary],
        'Examples': [summary[k]['num_examples'] for k in summary]
    })

    print("\n===== EVALUATION SUMMARY =====")
    print(df_summary)

    # Visualize results
    plt.figure(figsize=(12, 6))

    bar_width = 0.15
    r1 = np.arange(len(summary))
    r2 = [x + bar_width for x in r1]
    r3 = [x + bar_width for x in r2]
    r4 = [x + bar_width for x in r3]
    r5 = [x + bar_width for x in r4]

    plt.bar(r1, df_summary['Avg BLEU-1'], width=bar_width, label='BLEU-1')
    plt.bar(r2, df_summary['Avg BLEU-2'], width=bar_width, label='BLEU-2')
    plt.bar(r3, df_summary['Avg BLEU-4'], width=bar_width, label='BLEU-4')
    plt.bar(r4, df_summary['Avg Keyword Match'], width=bar_width, label='Keyword Match')
    plt.bar(r5, df_summary['Avg CodeBLEU'], width=bar_width, label='CodeBLEU')

    plt.xlabel('Translation Direction', fontweight='bold')
    plt.ylabel('Score', fontweight='bold')
    plt.xticks([r + bar_width*2 for r in range(len(summary))], df_summary['Translation'])
    plt.title('Code Translation Performance Metrics')
    plt.legend()
    plt.ylim(0, 1.0)
    plt.tight_layout()
    plt.savefig('code_translation_metrics.png')
    plt.show()

    return df_summary

# Step 8: Save results
def save_results(results, summary, output_dir='results'):
    """Save evaluation results and summary."""
    os.makedirs(output_dir, exist_ok=True)

    # Save detailed results
    with open(f'{output_dir}/detailed_results.json', 'w') as f:
        json.dump(results, f, indent=2)

    # Save summary
    summary.to_csv(f'{output_dir}/summary.csv', index=False)

    print(f"\nResults saved to {output_dir}/")

# Step 9: Main evaluation function
def main():
    print("Starting LLM Code Translation Evaluation...")

    # Load model and tokenizer
    print("\nLoading model and tokenizer...")
    model, tokenizer = load_model()

    # Prepare dataset
    print("\nPreparing dataset...")
    dataset = load_dataset_from_file('/content/all_languages_aligned.csv')

    # Alternatively, load your own dataset
    # dataset = load_dataset_from_file('your_dataset.json')

    # Run evaluations
    print("\nRunning evaluations...")
    results = evaluate_translations(model, tokenizer, dataset)

    # Analyze results
    print("\nAnalyzing results...")
    summary = analyze_results(results)

    # Save results
    save_results(results, summary)

    print("\nEvaluation completed!")

# Run the main function
if __name__ == "__main__":
    main()